In [ ]:
from IPython.display import display, Math
def solver(H, order):
    H_expanded = H.domain_expansion()[0]
    H_0 = H_expanded.group_by_order()[0]
    H_final = deepcopy(H_0)
    Bk = 0

    S = 0

    for k in range(1, order+1):
        print(f"Order {k}")
        H_below_k = sum(H_expanded.group_by_order().get(j, 0) for j in range(k+1))
        Vk = H_below_k.group_by_diagonal()[False].group_by_order().get(k, 0)
        S_k, symbols_sk = get_ansatz(Vk + Bk)
        solution_sk = dict(zip(symbols_sk, [0 for _ in symbols_sk]))
        #display(Bk)
        for key, value in ((H_0 | S_k) + (Vk + Bk)).group_by_infinite_terms().items():
            print("Im here 1")
            mats = sp.Add(*[sp.Mul(*map(lambda x: x.representation , term.info["coeff"])) * term.mat_representation["finite"] for term in value.terms])
            solution_sk.update(sp.solve(mats, symbols_sk))
        
        S_k_solved = 0
        for term in S_k.terms:
            print("Im here 2")
            term.mat_representation["finite"] = term.mat_representation["finite"].subs(solution_sk)
            if sum(sp.Abs(term.mat_representation["finite"])) == 0:
                continue
            S_k_solved += term
        S += S_k_solved
        display(S)
        tmp_H = float(1 / sp.factorial(k)) * H_below_k.nested_commutator(S, k)
        H_final += tmp_H.group_by_diagonal().get(True, Expression(Term(Coefficient(0)))).group_by_order().get(k, 0)
        display(H_final)
        Bk = tmp_H.group_by_diagonal()[False].group_by_order()[k+1]
    
    return H_final


# Trying out sympy module

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.sympy.classes import *
from Modules.sympy.utils import *
from sympy.physics.quantum import Dagger

In [3]:
sx = RDOperator("sigma_x", subspace = "spin", dim = 2, matrix=sp.Matrix([[0,1],[1,0]]))
sz = RDOperator("sigma_z", subspace = "spin", dim = 2, matrix=sp.Matrix([[1,0],[0,-1]]))


In [24]:
names = ["sigma_x", "sigma_z"]
subspaces = ["spin", "spin"]
dims = [2, 2, 2]
matrices = [sp.Matrix([[0,1],[1,0]]), sp.Matrix([[1,0],[0,-1]])]
''
sx, sz = get_finite_operators(names, subspaces, dims, matrices)
a = RDBoson("a", subspace ="b")
ad = RDBoson("{a^\\dagger}", subspace ="b", is_annihilation=False)

hbar = RDsymbol("hbar", order = 0)
omega = RDsymbol("omega", order = 0)
Omega_z = RDsymbol("Omega_z", order = 0)
g = RDsymbol("g", order = 1)
f = RDsymbol("f", order = 2)

H = hbar * omega * ad * a + hbar * Omega_z * sp.Rational(1,2) * sz + hbar * g * (a + ad) * sx
H

hbar*Omega_z*sigma_z/2 + hbar*g*(a + {a^\dagger})*sigma_x + hbar*omega*{a^\dagger}*a

In [25]:
H_ordered = group_by_order(H)
H0 = H_ordered.get(0, 0)
Vk = H_ordered.get(1, 0)
H0 + Vk

hbar*Omega_z*sigma_z/2 + hbar*g*a*sigma_x + hbar*g*{a^\dagger}*sigma_x + hbar*omega*{a^\dagger}*a

In [68]:
def make_matrix(name='a', n=2):
    symbolss = [[sp.symbols(f'{name}_'+ '{' +f'{i}{j}' + '}') for j in range(n)] for i in range(n)]
    s = sp.Matrix(symbolss)
    return s, s.reshape(1,n*n)

In [69]:
S1 = RDOperator('S1', subspace='finite', dim=2, matrix=make_matrix('S1', 2)[0])
S2 = RDOperator('S2', subspace='finite', dim=2, matrix=make_matrix('S2', 2)[0])
S1_total = S1 * a + S2 * ad
equations_to_solve = group_by_infinite_operators(expand_commutator(Commutator(H0, S1_total)).doit() + Vk)
equations = []
for _, value in equations_to_solve.items():
    equations.append(value)
matrices = [get_matrix(eq) for eq in equations if eq != 0]

In [70]:
matrices[0]

Matrix([
[                                -S1_{00}*hbar*omega, S1_{01}*hbar*Omega_z - S1_{01}*hbar*omega + hbar*g],
[-S1_{10}*hbar*Omega_z - S1_{10}*hbar*omega + hbar*g,                                -S1_{11}*hbar*omega]])

In [71]:
matrices[1] 

Matrix([
[                                 S2_{00}*hbar*omega, S2_{01}*hbar*Omega_z + S2_{01}*hbar*omega + hbar*g],
[-S2_{10}*hbar*Omega_z + S2_{10}*hbar*omega + hbar*g,                                 S2_{11}*hbar*omega]])

In [113]:
sp.solve(list(matrices[0].reshape(4,1))[1], tuple(make_matrix('S1', 2)[0])[1])

ValueError: 
Can't calculate derivative wrt Omega_z.

In [117]:
Omega_z.order

0

In [109]:
tuple(make_matrix('S1', 2)[0])[0]

S1_{00}